## Семинар 6: "Архитектуры и transfer learning"

ФИО:  Дегтярев Роман Олегович

In [1]:
import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import MNIST
from torchvision import transforms
from torchvision import models
import pandas as pd
import os
from skimage import io, transform
from torchvision import models,transforms,datasets
import time


%matplotlib inline

На этом семинаре мы попробуем улучшить результаты, полученные на занятии номер 3 с помощью Transfer Learning

### Часть 1: Inference

Пропустите картинки с лицами через предобученную на Imagenet "сверточную часть" AlexNet или VGG сетей и обучите на полученных данных один полносвязный слой.

In [2]:
#зададим папки
train_landmarks_path = 'dataset/train/face_landmarks.csv'
train_path = 'dataset/train'
test_landmarks_path = 'dataset/test/face_landmarks.csv'
test_path = 'dataset/test'

In [3]:
#зададим модель
model_vgg = models.vgg16(pretrained=True)

In [7]:
for param in model_vgg.parameters():
    param.requires_grad = False
model_vgg.fc = nn.Linear(1000,136)

In [8]:
#считаем данные
#для этого зададим класс, отнаследованный от dataset. 
# у этого класса будет методы: инициализация и вернуть картинку
class faces(Dataset):
    def __init__(self, csv_file, root_dir, transform= None):
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.landmarks_frame.ix[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.ix[idx, 1:].as_matrix().astype('float')
        landmarks = landmarks.reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']
 
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        #image = image.transpose((0, 1))
        return {'image': torch.from_numpy(image),
                'landmarks': torch.from_numpy(landmarks)}

In [9]:
train = faces(csv_file=train_landmarks_path,
                             root_dir= train_path, 
                             transform=transforms.Compose([ToTensor()])
             )

In [16]:
dataloader = DataLoader(train, batch_size=40,
                        shuffle=True)

In [25]:
model_vgg

VGG (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU (inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU (inplace)
    (4): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU (inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU (inplace)
    (9): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU (inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU (inplace)
    (16): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), pa

In [24]:
loss_fn = torch.nn.MSELoss(size_average=True)

learning_rate = 0.01
error = []
error_train = []
optimizer =  torch.optim.SGD(filter(lambda p: p.requires_grad, model_vgg.parameters()), lr=1e-3)
for t in range(10):
    for T in dataloader:
        X = T['image'].resize_(T['image'].size(0),3, 96, 96).float()
        Y = T['landmarks'].view(T['landmarks'].size(0), -1).float()
        x = Variable(X, requires_grad=False)
        y = Variable(Y, requires_grad=False)
        
        y_pred = model_vgg(x)

        loss =  loss_fn(y_pred, y)
        error.append(loss.data[0])
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
    print ('epoch %d, meanerror %f' % (t,  np.array(error).mean()))
    error_train.append(np.array(error).mean())

RuntimeError: size mismatch, m1: [40 x 4608], m2: [25088 x 4096] at /Users/soumith/miniconda2/conda-bld/pytorch_1503975203422/work/torch/lib/TH/generic/THTensorMath.c:1293

### Часть 2 (опциональная)

Попробуйте обучить одну из архитектур на датасете. Сравните результаты с пунктом 1  

#### Feedback (опционально)

Здесь вы можете оставить список опечаток из лекции или семинара:

Здесь вы можете оставить комментарии по лекции или семинару: